<a href="https://colab.research.google.com/github/noobsterguy/assignment_ml_1/blob/master/test_request.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabajo sobre Natural Language Processing

---

> Presentado por: Ivancovish Julian Huancco

## Primera parte - Análisis de textos

Primero, importamos las librerias necesarias:

In [499]:
import spacy
import en_core_web_sm
from spacy.lang.en import English
!python -m spacy download en_core_web_sm

import pandas as pd
import numpy as np

# Para el stemming
import nltk
nltk.download('punkt')
nltk.download('vader_lexicon')

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.porter import *
#spacy no lo considera por que asumen que no lo considera util

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


#### Nota:

> Con motivos de automatización general para futuros projectos, se uso una API bastante efectiva de GOODREADS.

In [ ]:
!git clone https://github.com/pirmoradian/goodreads

In [501]:
!ls

apikey.py.enc  dist  goodreads		 MANIFEST.in  requirements.txt	tests
build	       docs  goodreads.egg-info  README.rst   setup.py


In [ ]:
cd goodreads/

In [ ]:
!pip install -r requirements.txt

In [ ]:
!python setup.py install

In [505]:
from goodreads import client
gc = client.GoodreadsClient('jR8XaOotmd9aVPwHk0E7Fg', 'a1LEOVzkv1j1oitKzElbqgpdVXiijTxqcQ2nlr8')

### Importando el texto que se requiere analisar 

El texto que se quiere analisisar es una pequeña descripción del Libro que tiene como código 5552 publicado en GOODREADS.
https://www.goodreads.com/book/show/5552.QED

Verificamos con el siguiente script

In [506]:
book = gc.book(5552)
book

QED: The Strange Theory of Light and Matter

Ahora, importamos el texto que se desea analisar

In [507]:
texto=book.description[:]
texto

'Famous the world over for the creative brilliance of his insights into the physical world, Nobel Prize-winning physicist Richard Feynman also possessed an extraordinary talent for explaining difficult concepts to the nonscientist. QED--the edited version of four lectures on quantum electrodynamics that Feynman gave to the general public at UCLA as part of the Alix G. Mautner Memorial Lecture series--is perhaps the best example of his ability to communicate both the substance and the spirit of science to the layperson.<br /><br />The focus, as the title suggests, is quantum electrodynamics (QED), the part of the quantum theory of fields that describes the interactions of the quanta of the electromagnetic field-light, X rays, gamma rays--with matter and those of charged particles with one another. By extending the formalism developed by Dirac in 1933, which related quantum and classical descriptions of the motion of particles, Feynman revolutionized the quantum mechanical understanding 

### TOKENIZACIÓN

Ahora realizamos el proceso de tokenización

In [508]:
# Importar spaCy y cargar la biblioteca de idiomas
#import spacy
nlp = en_core_web_sm.load()

# Crea un objeto Doc
doc = nlp(texto)

# Imprime cada token por separado
for token in doc:
    print(token.text, token.pos_, token.dep_)

Famous ADJ advcl
the DET det
world NOUN dobj
over ADP advmod
for ADP prep
the DET det
creative ADJ amod
brilliance NOUN pobj
of ADP prep
his DET poss
insights NOUN pobj
into ADP prep
the DET det
physical ADJ amod
world NOUN pobj
, PUNCT punct
Nobel PROPN compound
Prize PROPN npadvmod
- PUNCT punct
winning VERB amod
physicist NOUN compound
Richard PROPN compound
Feynman PROPN nsubj
also ADV advmod
possessed VERB ROOT
an DET det
extraordinary ADJ amod
talent NOUN dobj
for ADP prep
explaining VERB pcomp
difficult ADJ amod
concepts NOUN dobj
to ADP prep
the DET det
nonscientist NOUN pobj
. PUNCT punct
QED PROPN nsubj
-- PUNCT punct
the DET det
edited ADJ amod
version NOUN appos
of ADP prep
four NUM nummod
lectures NOUN pobj
on ADP prep
quantum ADJ compound
electrodynamics NOUN pobj
that DET dobj
Feynman PROPN nsubj
gave VERB relcl
to ADP dative
the DET det
general ADJ amod
public NOUN pobj
at ADP prep
UCLA PROPN pobj
as SCONJ prep
part NOUN pobj
of ADP prep
the DET det
Alix PROPN compound


Creaando un objeto Doc y explora tokens.

In [509]:
# Create a Doc object and explore tokens y contando los token
doc = nlp(texto)

for token in doc:
    print(token.text, end=' | ')

Famous | the | world | over | for | the | creative | brilliance | of | his | insights | into | the | physical | world | , | Nobel | Prize | - | winning | physicist | Richard | Feynman | also | possessed | an | extraordinary | talent | for | explaining | difficult | concepts | to | the | nonscientist | . | QED | -- | the | edited | version | of | four | lectures | on | quantum | electrodynamics | that | Feynman | gave | to | the | general | public | at | UCLA | as | part | of | the | Alix | G. | Mautner | Memorial | Lecture | series | -- | is | perhaps | the | best | example | of | his | ability | to | communicate | both | the | substance | and | the | spirit | of | science | to | the | layperson.<br | /><br | />The | focus | , | as | the | title | suggests | , | is | quantum | electrodynamics | ( | QED | ) | , | the | part | of | the | quantum | theory | of | fields | that | describes | the | interactions | of | the | quanta | of | the | electromagnetic | field | - | light | , | X | ra

Contando numero de tokens y oraciones separadas

In [510]:
sents = [sent for sent in doc.sents]
print('el número de tokens es de:',len(doc))
print('el número de oraciones sepradas es de:',len(sents))

el número de tokens es de: 285
el número de oraciones sepradas es de: 6


Entidades con nombre

In [511]:
doc.ents

(Nobel Prize,
 Richard Feynman,
 QED,
 four,
 Feynman,
 UCLA,
 Dirac,
 1933,
 Feynman,
 Feynman,
 QED,
 Feynman,
 Feynman,
 QED)

A pesar de ser un texto formal, tambien es posible encontrar entidades con nombre tales como abreviaciones, fecha y nombres propios. Una visualización de sus caracteristicas seria tal como se muestra a continuación.

In [512]:
for ent in doc.ents:
    print(ent.text+' - '+ent.label_+' - '+str(spacy.explain(ent.label_)))

Nobel Prize - WORK_OF_ART - Titles of books, songs, etc.
Richard Feynman - PERSON - People, including fictional
QED - ORG - Companies, agencies, institutions, etc.
four - CARDINAL - Numerals that do not fall under another type
Feynman - NORP - Nationalities or religious or political groups
UCLA - ORG - Companies, agencies, institutions, etc.
Dirac - PERSON - People, including fictional
1933 - DATE - Absolute or relative dates or periods
Feynman - ORG - Companies, agencies, institutions, etc.
Feynman - ORG - Companies, agencies, institutions, etc.
QED - ORG - Companies, agencies, institutions, etc.
Feynman - NORP - Nationalities or religious or political groups
Feynman - ORG - Companies, agencies, institutions, etc.
QED - ORG - Companies, agencies, institutions, etc.


Probablemente falte implementar una caracteristica importante en cuanto a los nombres propios que pueden volverse objetos o adjetivos, tales como "Feynman Diagrams" que deberia ser lo correcto.

### STEMMING

In [513]:
#Necesitaremos de la siguiente clase:
p_stemmer = PorterStemmer()

Para facilitar el analisis de stemming en una sentencia creamos una funcion.

In [514]:
def stemming(texto):
    token_words=word_tokenize(texto)
    token_words
    stem_texto=[]
    for word in token_words:
        stem_texto.append(p_stemmer.stem(word))
        stem_texto.append(" ")
    return "".join(stem_texto)

print(texto)
print("-Sentencias STEMMING")
x=stemming(texto)
print(x)

Famous the world over for the creative brilliance of his insights into the physical world, Nobel Prize-winning physicist Richard Feynman also possessed an extraordinary talent for explaining difficult concepts to the nonscientist. QED--the edited version of four lectures on quantum electrodynamics that Feynman gave to the general public at UCLA as part of the Alix G. Mautner Memorial Lecture series--is perhaps the best example of his ability to communicate both the substance and the spirit of science to the layperson.<br /><br />The focus, as the title suggests, is quantum electrodynamics (QED), the part of the quantum theory of fields that describes the interactions of the quanta of the electromagnetic field-light, X rays, gamma rays--with matter and those of charged particles with one another. By extending the formalism developed by Dirac in 1933, which related quantum and classical descriptions of the motion of particles, Feynman revolutionized the quantum mechanical understanding o

Como se puede ver efectivamente en algunas palabras falla el stimming

### LEMMATIZACION

In [515]:
doc1 = nlp(texto)

for token in doc1:
    print(token.text, '\t', token.pos_, '\t', token.lemma_)

Famous 	 ADJ 	 famous
the 	 DET 	 the
world 	 NOUN 	 world
over 	 ADP 	 over
for 	 ADP 	 for
the 	 DET 	 the
creative 	 ADJ 	 creative
brilliance 	 NOUN 	 brilliance
of 	 ADP 	 of
his 	 DET 	 -PRON-
insights 	 NOUN 	 insight
into 	 ADP 	 into
the 	 DET 	 the
physical 	 ADJ 	 physical
world 	 NOUN 	 world
, 	 PUNCT 	 ,
Nobel 	 PROPN 	 Nobel
Prize 	 PROPN 	 Prize
- 	 PUNCT 	 -
winning 	 VERB 	 win
physicist 	 NOUN 	 physicist
Richard 	 PROPN 	 Richard
Feynman 	 PROPN 	 Feynman
also 	 ADV 	 also
possessed 	 VERB 	 possess
an 	 DET 	 an
extraordinary 	 ADJ 	 extraordinary
talent 	 NOUN 	 talent
for 	 ADP 	 for
explaining 	 VERB 	 explain
difficult 	 ADJ 	 difficult
concepts 	 NOUN 	 concept
to 	 ADP 	 to
the 	 DET 	 the
nonscientist 	 NOUN 	 nonscientist
. 	 PUNCT 	 .
QED 	 PROPN 	 QED
-- 	 PUNCT 	 --
the 	 DET 	 the
edited 	 ADJ 	 edited
version 	 NOUN 	 version
of 	 ADP 	 of
four 	 NUM 	 four
lectures 	 NOUN 	 lecture
on 	 ADP 	 on
quantum 	 ADJ 	 quantum
electrodynamics 	 NOUN 	 electro

## Segunda Parte - Analisis de sentimientos

Para esta sección vamos a analisar 10 reviews del libro.

In [516]:
# una pequeña verificaci{on de cuantos reviews hay
n_reviews=book.text_reviews_count
print("El número de reviews es:",n_reviews)

El número de reviews es: 338


Más información

In [517]:
gc.book_review_stats([book.isbn])

[{'average_rating': '4.24',
  'id': 5552,
  'isbn': '0691024170',
  'isbn13': '9780691024172',
  'ratings_count': 13971,
  'reviews_count': 37059,
  'text_reviews_count': 338,
  'work_ratings_count': 15585,
  'work_reviews_count': 40927,
  'work_text_reviews_count': 497}]

Probamos para una review

In [518]:
review_id = 694911068
review = gc.review(review_id)
review

review [694911068]

In [519]:
example=review.body
example

"If science ever had a poet. it's this man. wait. let's see if i can do this - - Richard Feynman did to Quantum-electrodynamics what Emily Dickinson did to poetry. Got rid of the elaborate insanity it was dressed in, refined into a melodic phrase, and deepened its meaning."

Ahora intentaremos trabajar con almenos 5 reviews aleatorias para mostrar parcialidad. revies de la pág , pág 5, pág 8, pág 9 y Pág 10.

Y convertiremos en un Dataframe

In [520]:
filas=[gc.review(694911068).body,gc.review(3152096503).body,gc.review(459009298).body,gc.review(2260565490).body,gc.review(1313928038).body,gc.review(3389139947).body,gc.review(1756461796).body,gc.review(130223210).body,gc.review(821588288).body,gc.review(42596292).body]

In [521]:
Puntuacion=[gc.review(694911068).rating,gc.review(3152096503).rating,gc.review(459009298).rating,gc.review(2260565490).rating,gc.review(1313928038).rating,gc.review(3389139947).rating,gc.review(1756461796).rating,gc.review(130223210).rating,gc.review(821588288).rating,gc.review(42596292).rating]

In [522]:
data = pd.DataFrame(list(zip(Puntuacion,filas)), columns=['Rating','reviews'])
data

,Rating,reviews
0,5,If science ever had a poet. it's this man. wai...
1,3,I can't say that I know mich physics than I've...
2,3,My biggest mistake here was reading this in sm...
3,1,"Maybe I shouldn’t be surprised, but this just ..."
4,4,"Like the preface of the book explains, this is..."
5,5,Just brilliant! Probably the best piece of non...
6,5,This is a very intuitive explain in some basic...
7,5,Despite the technical nature of his explanatio...
8,5,A masterpiece of popular science. Feynman did ...
9,5,"Perfect for what it is, which is a concise and..."


In [523]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Rating   10 non-null     object
 1   reviews  10 non-null     object
dtypes: object(2)
memory usage: 288.0+ bytes


A partir de esto realizamos un analisis de sentimiento.

- Primero, Podemos crear una nueva columna que almacene la longitud de la cadena de cada muestra de texto y luego ordenar las filas de DataFrame en orden ascendente de la longitud del texto.


In [524]:
data['len'] = data['reviews'].str.len()  # Almacene la longitud de la cadena de cada muestra
data = data.sort_values(['len'], ascending=True)
data

,Rating,reviews,len
3,1,"Maybe I shouldn’t be surprised, but this just ...",219
9,5,"Perfect for what it is, which is a concise and...",220
0,5,If science ever had a poet. it's this man. wai...,272
7,5,Despite the technical nature of his explanatio...,282
5,5,Just brilliant! Probably the best piece of non...,291
2,3,My biggest mistake here was reading this in sm...,300
4,4,"Like the preface of the book explains, this is...",300
8,5,A masterpiece of popular science. Feynman did ...,300
1,3,I can't say that I know mich physics than I've...,305
6,5,This is a very intuitive explain in some basic...,310


- Segundo, Para mayor facilidad Utilizaremos un criterio aceptacion para obtener resultados de forma binaria.

In [525]:
data['Rating'] = data['Rating'].astype(int)

In [526]:
data['Rating'] = data['Rating'].apply(lambda c: 'pos' if c >=3 else 'neg')

In [527]:
data

,Rating,reviews,len
3,neg,"Maybe I shouldn’t be surprised, but this just ...",219
9,pos,"Perfect for what it is, which is a concise and...",220
0,pos,If science ever had a poet. it's this man. wai...,272
7,pos,Despite the technical nature of his explanatio...,282
5,pos,Just brilliant! Probably the best piece of non...,291
2,pos,My biggest mistake here was reading this in sm...,300
4,pos,"Like the preface of the book explains, this is...",300
8,pos,A masterpiece of popular science. Feynman did ...,300
1,pos,I can't say that I know mich physics than I've...,305
6,pos,This is a very intuitive explain in some basic...,310


Las muestras con palabras claramente polares, como "Perfect" y "marterprice" ofrecerían un mayor contexto a un clasificador de sentimientos y para palabras que suenan neutrales (como "maybe" o "mistake") 



Esto lo comprobamos con la funcion polarity

In [528]:
sid.polarity_scores(data.loc[0]['reviews'])

{'compound': -0.5719, 'neg': 0.079, 'neu': 0.921, 'pos': 0.0}

In [529]:
data.loc[0]['reviews']

"If science ever had a poet. it's this man. wait. let's see if i can do this - - Richard Feynman did to Quantum-electrodynamics what Emily Dickinson did to poetry. Got rid of the elaborate insanity it was dressed in, refined into a melodic phrase, and deepened its meaning."

In [530]:
data.loc[0]['Rating']

'pos'

El tener un composición neutral+positiva--- y un compound muy positivo resultaria como clasificación positiva


In [531]:
# aplicar a cada elemento y pegarlo en el dataframe (importante)
data['scores'] = data['reviews'].apply(lambda review: sid.polarity_scores(review))

data.head()

,Rating,reviews,len,scores
3,neg,"Maybe I shouldn’t be surprised, but this just ...",219,"{'neg': 0.0, 'neu': 0.73, 'pos': 0.27, 'compou..."
9,pos,"Perfect for what it is, which is a concise and...",220,"{'neg': 0.057, 'neu': 0.641, 'pos': 0.302, 'co..."
0,pos,If science ever had a poet. it's this man. wai...,272,"{'neg': 0.079, 'neu': 0.921, 'pos': 0.0, 'comp..."
7,pos,Despite the technical nature of his explanatio...,282,"{'neg': 0.0, 'neu': 0.788, 'pos': 0.212, 'comp..."
5,pos,Just brilliant! Probably the best piece of non...,291,"{'neg': 0.0, 'neu': 0.759, 'pos': 0.241, 'comp..."


Agregamos las composiciones.

In [532]:
data['compound']  = data['scores'].apply(lambda score_dict: score_dict['compound'])

data.head()

,Rating,reviews,len,scores,compound
3,neg,"Maybe I shouldn’t be surprised, but this just ...",219,"{'neg': 0.0, 'neu': 0.73, 'pos': 0.27, 'compou...",0.8662
9,pos,"Perfect for what it is, which is a concise and...",220,"{'neg': 0.057, 'neu': 0.641, 'pos': 0.302, 'co...",0.8932
0,pos,If science ever had a poet. it's this man. wai...,272,"{'neg': 0.079, 'neu': 0.921, 'pos': 0.0, 'comp...",-0.5719
7,pos,Despite the technical nature of his explanatio...,282,"{'neg': 0.0, 'neu': 0.788, 'pos': 0.212, 'comp...",0.8910
5,pos,Just brilliant! Probably the best piece of non...,291,"{'neg': 0.0, 'neu': 0.759, 'pos': 0.241, 'comp...",0.9200


Y su score respectivo

In [533]:
data['comp_score'] = data['compound'].apply(lambda c: 'pos' if c >=0 else 'neg')

data

,Rating,reviews,len,scores,compound,comp_score
3,neg,"Maybe I shouldn’t be surprised, but this just ...",219,"{'neg': 0.0, 'neu': 0.73, 'pos': 0.27, 'compou...",0.8662,pos
9,pos,"Perfect for what it is, which is a concise and...",220,"{'neg': 0.057, 'neu': 0.641, 'pos': 0.302, 'co...",0.8932,pos
0,pos,If science ever had a poet. it's this man. wai...,272,"{'neg': 0.079, 'neu': 0.921, 'pos': 0.0, 'comp...",-0.5719,neg
7,pos,Despite the technical nature of his explanatio...,282,"{'neg': 0.0, 'neu': 0.788, 'pos': 0.212, 'comp...",0.8910,pos
5,pos,Just brilliant! Probably the best piece of non...,291,"{'neg': 0.0, 'neu': 0.759, 'pos': 0.241, 'comp...",0.9200,pos
2,pos,My biggest mistake here was reading this in sm...,300,"{'neg': 0.03, 'neu': 0.886, 'pos': 0.084, 'com...",0.4854,pos
4,pos,"Like the preface of the book explains, this is...",300,"{'neg': 0.053, 'neu': 0.778, 'pos': 0.17, 'com...",0.7647,pos
8,pos,A masterpiece of popular science. Feynman did ...,300,"{'neg': 0.04, 'neu': 0.836, 'pos': 0.124, 'com...",0.4684,pos
1,pos,I can't say that I know mich physics than I've...,305,"{'neg': 0.0, 'neu': 0.843, 'pos': 0.157, 'comp...",0.8221,pos
6,pos,This is a very intuitive explain in some basic...,310,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,pos


Hasta aqui hemos logrado identificar el score de compon identificacando los valores positivos y negativos

### Medidas de ajuste

#### Accuracy

In [534]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [535]:
accuracy_score(data['Rating'],data['comp_score'])

0.8

In [536]:
print(confusion_matrix(data['Rating'],data['comp_score']))


[[0 1]
 [1 8]]


#### Split the data into train & test sets:

In [537]:
from sklearn.model_selection import train_test_split

X = data['reviews']
y = data['Rating']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

Construir un pipeline

In [538]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

#podemos aplicar todo lo que se aprendio
# Naïve Bayes:
text_clf_nb = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', MultinomialNB()),
])

# Linear SVC:
text_clf_lsvc = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC()),
])

In [539]:
text_clf_nb.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

Revisando Resultados

In [540]:
# Form a prediction set
predictions = text_clf_nb.predict(X_test)

In [541]:
# Report the confusion matrix
from sklearn import metrics
print(metrics.confusion_matrix(y_test,predictions))

[[0 1]
 [0 3]]


In [542]:
# Calculo de accuracy
print(metrics.accuracy_score(y_test,predictions))

0.75


## Conclusiones

- Primera parte - Análisis de textos

> - De alguna manera se logró automatizar la extracción de datos para facilitar el análisis de textos y reviews. El tópico elegido para el análisis fue de libros de la web GOODREADS, Se utilizó una API que fue de gran ayuda https://github.com/pirmoradian/goodreads.

> - Se logro análisar un texto menor a 300 palabras de un libro de ciencia "QED: The Strange Theory of Light and Matter" de "Richard Feynman", verificando la Tokenización, Stemming y lemmatización, donde  dos primeros cumplen el proposito. Sin embargo, en cuanto a la lemmatización hay palabras que no vuelven a su nucleo o lemma principal.

- Segunda parte - Analsis de sentimientos

> - Se tomo 10 reviews del mismo libro. y se realizo un analisis de sentimientos, cuya exactitud cálculada fue de: 0,8


> - Se escogio una clasificación de positivos y negativos para el "Rating" tomando como valor de referencia para positivos >= 3 (de 5) y negativos menores a 3 de tal manera que se ajuste a nuestra predicción.

> - El pipeline escogido para el analisis fue el de Naive bayes. cuya exactitud cálculada fue de 0,75.
